# Generation of files for media preparation using biomek

This notebook generates several files used in an automated media optimization project. Given a file with stock concentrations and a file with target concentrations (recommended for example by ART), this notebooks generates transfer volumes needed to achieve those target concentrations. Transfer volumes are provided as a table for all components and destination wells, but also as files required for a Biomek run. The notebook also generates a file with instructions on how to prepare a plate with stock solutions.

## Inputs and outputs

### Required files to run this notebook:
   - `standard_recipe_concentrations.csv` - this file will not change over the course of a particular project
   
   - `stock_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/DBTLX/target_concentrations.csv` - this is an output from an ART run and it will change at every DBTL cycle
   
   - `24-well_stock_plate_high.csv`, `24-well_stock_plate_low.csv`, `24-well_stock_plate_fresh.csv` - instructions on how to prepare the source plates
     

### Files generated by running this notebook:
   
   - `dest_volumes.csv` - volumes of all components in each of the destination wells
   
   - `media_descriptions.csv` - complete list of media designs in terms of concentrations for each each well
   
   
   
The files that need to be uploaded to the Biomek, following this particular order:
   
   - `P200_water.csv`
   
   - `P20_water.csv`
         
   - `P20_kan.csv` 
   
   - `P200_components.csv` 
   
   - `P20_components.csv` 
  
   - `P20_culture.csv` 
   
   
Note that all these different files are needed as biomek needs a separate one for different types of source/destination plate and tips. Also, it cannot follow the order of operations within a file so we have to enforce an order using different files (e.g. first a file with kan transfers, then a file with components transfers).
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [1]:
import os
import sys
sys.path.append('../../media_compiler')

import pandas as pd
import numpy as np

from core import find_volumes, find_volumes_bulk


## User parameters

In [2]:
CYCLE = 7

user_params = {
    'stock_conc_file': '../data/stock_concentrations_extended.csv',
    'standard_media_file': '../data/standard_recipe_concentrations_extended.csv',
    'stock_plate_high_file': '../data/24-well_stock_plate_high_extended.csv',
    'stock_plate_low_file': '../data/24-well_stock_plate_low_extended.csv',
    'stock_plate_fresh_file': '../data/24-well_stock_plate_fresh_extended.csv',
    'target_conc_file': f'../data/DBTL{CYCLE}/target_concentrations.csv',
    'output_path': f'../data/DBTL{CYCLE}',  # Path for output files
    'well_volume': 1500,            # Total volume of the media content in the destination well
    'tips': ['f20', 'f200'],        # Choose available tips from f20, s20, f50, s50, f200, s200
    'min_transfer_volume': 5.,      # Minimal transfer volume of the liquid handler
    'culture_factor': 100           # Dilution factor for culture, e.g. 100x, 1000x

}

Setup tips

In [3]:
df_tips = pd.DataFrame([['f20', 36],
                        ['s20', 72],
                        ['f50', 45],
                        ['s50', 81],
                        ['f200', 190],
                        ['s200', 171]],
                       columns=['Tips', 'Max Volume'],
                       ).set_index('Tips')


In [4]:
max_tip_volume = max([df_tips.loc[tip]['Max Volume'] for tip in user_params['tips']])


Load the standard media recipe

In [5]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

In [6]:
# df_stand

Load the stock concentrations and stock plate layouts:

In [7]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")
df_stock_plate_high = pd.read_csv(user_params['stock_plate_high_file'])
df_stock_plate_low = pd.read_csv(user_params['stock_plate_low_file'])
df_stock_plate_fresh = pd.read_csv(user_params['stock_plate_fresh_file'])


In [8]:
df_stock_plate_fresh

,Well,Component,Concentration
0,A1,Culture,NaN
1,B1,FeSO4[mM],0.3
2,C1,FeSO4[mM],6.0


In [9]:
df_stock

,Low Concentration,High Concentration,Dilution Factor
Component,,,
MOPS[mM],2000.0000,2000.000,1.0
Tricine[mM],400.0000,400.000,1.0
H3BO3[mM],0.1200,4.800,40.0
Glucose[mM],3000.0000,3000.000,1.0
K2SO4[mM],3.0000,60.000,20.0
K2HPO4[mM],79.2000,792.000,10.0
FeSO4[mM],0.3000,6.000,20.0
NH4Cl[mM],1920.0000,1920.000,1.0
MgCl2[mM],7.8000,156.000,20.0


Reformat values from string to float in the `df_stock` dataframe:

In [9]:
# df_stock.loc['Kan'] = [300., 300., 1.]
# df_stock["High Concentration"] = df_stock["High Concentration"].astype(float)
# df_stock["Low Concentration"] = df_stock["Low Concentration"].astype(float)

In [10]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0000,2000.0000,1.0
Tricine,400.0000,400.0000,1.0
H3BO3,0.1200,2.4000,20.0
Glucose,3000.0000,3000.0000,1.0
K2SO4,8.7000,43.5000,5.0
K2HPO4,79.2000,396.0000,5.0
FeSO4,0.3000,6.0000,20.0
NH4Cl,1904.0001,1904.0001,1.0
MgCl2,15.6000,15.6000,1.0


## Read target concentrations

Read ART suggested target concentrations. Note that those are only for components which are being explored (not those that are kept constant). 

In [10]:
target_conc_file = user_params['target_conc_file']
df_target_conc = pd.read_csv(target_conc_file, index_col=0)
df_target_conc.head()

,H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],CaCl2[mM],Label,OD340_pred,OD340_std
Well,,,,,,,,,,,,,,,,,
A1,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,LHS,0.566117,0.094277
B1,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,LHS,0.566117,0.094277
C1,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,LHS,0.566117,0.094277
D1,0.073964,197.001031,0.805718,8.730541,0.024996,34.480900,1.287242,959.379367,0.000295,0.001569,0.005873,0.013567,0.000728,0.016770,LHS,0.590909,0.097057
E1,0.073964,197.001031,0.805718,8.730541,0.024996,34.480900,1.287242,959.379367,0.000295,0.001569,0.005873,0.013567,0.000728,0.016770,LHS,0.590909,0.097057


### Add fixed components and antibiotic concentrations from the standard recipe to the target concentrations dataframe

Find fixed components as those from the standard recipe that are not listed in target concentration file, which is an output from ART

In [11]:
comp_var = df_target_conc.iloc[:, :-3].columns
comp_fixed = list(df_stand.drop(comp_var).index)
print('Fixed components: ')
for comp in comp_fixed:
    df_target_conc[comp] = df_stand.at[comp, 'Concentration']
    print(f'{comp}')

Fixed components: 
MOPS[mM]
Tricine[mM]
Kan[g/L]


Make sure the order of columns is the same as in the stock dataframe:

In [12]:
columns = df_stock.index
df_target_conc = df_target_conc[columns]

In [13]:
df_target_conc.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],CaCl2[mM],Kan[g/L]
Well,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,0.05
B1,40.0,4.0,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,0.05
C1,40.0,4.0,0.033485,157.760522,0.964123,6.992958,0.067892,23.233056,1.932010,1313.377579,0.000108,0.002725,0.004785,0.007395,0.001752,0.035968,0.05
D1,40.0,4.0,0.073964,197.001031,0.805718,8.730541,0.024996,34.480900,1.287242,959.379367,0.000295,0.001569,0.005873,0.013567,0.000728,0.016770,0.05
E1,40.0,4.0,0.073964,197.001031,0.805718,8.730541,0.024996,34.480900,1.287242,959.379367,0.000295,0.001569,0.005873,0.013567,0.000728,0.016770,0.05


Save a file with a complete list of components concentrations, describing the media (this file will lated be used to create experiment description file for EDD import).

In [19]:
final_conc_file = f"{user_params['output_path']}/media_descriptions.csv"
df_target_conc.drop(columns='Kan[g/L]').to_csv(final_conc_file)
df_target_conc.to_csv(final_conc_file)

## Calculate all transfer volumes

Also, create a dataframe with levels of stock concentrations needed to achieve those volumes, which will indicate from which source well the transfer should be made.

In [14]:
df_volumes, df_conc_level = find_volumes_bulk(
    df_stock=df_stock, 
    df_target_conc=df_target_conc,
    well_volume=user_params['well_volume'],
    min_tip_volume=user_params['min_transfer_volume'],
    culture_ratio=user_params['culture_factor'],
    verbose=1
)

Iteration 0:
Compoments small: Index(['(NH4)6Mo7O24[mM]'], dtype='object', name='Component')
Iteration 1:
Compoments small: Index(['(NH4)6Mo7O24[mM]'], dtype='object', name='Component')
Iteration 2:
Compoments small: Index(['(NH4)6Mo7O24[mM]'], dtype='object', name='Component')
Iteration 3:
Iteration 4:
Iteration 5:
Iteration 6:
Compoments small: Index(['H3BO3[mM]', 'CuSO4[mM]'], dtype='object', name='Component')
Iteration 7:
Compoments small: Index(['H3BO3[mM]', 'CuSO4[mM]'], dtype='object', name='Component')
Iteration 8:
Compoments small: Index(['H3BO3[mM]', 'CuSO4[mM]'], dtype='object', name='Component')
Iteration 9:
Compoments small: Index(['MnSO4[mM]', 'ZnSO4[mM]'], dtype='object', name='Component')
Iteration 10:
Compoments small: Index(['MnSO4[mM]', 'ZnSO4[mM]'], dtype='object', name='Component')
Iteration 11:
Compoments small: Index(['MnSO4[mM]', 'ZnSO4[mM]'], dtype='object', name='Component')
Iteration 12:
Compoments small: Index(['FeSO4[mM]', 'CaCl2[mM]'], dtype='object', name

In [15]:
df_volumes.iloc[-3:, :]

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],CaCl2[mM],Kan[g/L],Water
Well,,,,,,,,,,,,,,,,,,
D8,30.0,15.0,46.076622,10.349991,139.815406,25.288444,49.506986,7.312403,90.66942,52.92838,48.508558,47.516924,47.644527,46.643333,46.89867,0.0,5.0,775.840336
E8,30.0,15.0,46.076622,10.349991,139.815406,25.288444,49.506986,7.312403,90.66942,52.92838,48.508558,47.516924,47.644527,46.643333,46.89867,0.0,5.0,775.840336
F8,30.0,15.0,46.076622,10.349991,139.815406,25.288444,49.506986,7.312403,90.66942,52.92838,48.508558,47.516924,47.644527,46.643333,46.89867,0.0,5.0,775.840336


This is ok, as we don't have CaCl2 in the standard design.

In [16]:
df_conc_level.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],CaCl2[mM],Kan[g/L]
Well,,,,,,,,,,,,,,,,,
A1,high,high,high,high,high,high,high,high,high,high,low,high,high,high,high,high,high
B1,high,high,high,high,high,high,high,high,high,high,low,high,high,high,high,high,high
C1,high,high,high,high,high,high,high,high,high,high,low,high,high,high,high,high,high
D1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high
E1,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high,high


Add volumes for culture

In [17]:
df_volumes['Culture'] = user_params['well_volume'] / user_params['culture_factor']
df_volumes.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],CaCl2[mM],Kan[g/L],Water,Culture
Well,,,,,,,,,,,,,,,,,,,
A1,30.0,15.0,10.463932,78.880261,24.103073,13.244238,16.972882,18.150825,18.577018,525.351032,180.284537,11.355045,19.939178,11.553957,21.905091,17.983902,5.0,466.235029,15.0
B1,30.0,15.0,10.463932,78.880261,24.103073,13.244238,16.972882,18.150825,18.577018,525.351032,180.284537,11.355045,19.939178,11.553957,21.905091,17.983902,5.0,466.235029,15.0
C1,30.0,15.0,10.463932,78.880261,24.103073,13.244238,16.972882,18.150825,18.577018,525.351032,180.284537,11.355045,19.939178,11.553957,21.905091,17.983902,5.0,466.235029,15.0
D1,30.0,15.0,23.113867,98.500516,20.142961,16.535115,6.249088,26.938203,12.377325,383.751747,12.309804,6.537486,24.469353,21.198690,9.102220,8.385038,5.0,765.388588,15.0
E1,30.0,15.0,23.113867,98.500516,20.142961,16.535115,6.249088,26.938203,12.377325,383.751747,12.309804,6.537486,24.469353,21.198690,9.102220,8.385038,5.0,765.388588,15.0


Check if sum of all volumes is equal to total well volume

In [23]:
EPS = 0.000001
assert (np.sum(df_volumes.values, axis=1) - user_params['well_volume'] <= EPS).all(), 'Sum of all volumes is not equal to total well volume!' 

### Save volumes in the destination wells to a file

In [24]:
volumes_file = f"{user_params['output_path']}/dest_volumes.csv"
df_volumes.to_csv(volumes_file)

## Add minimal volumes needed in the source plates

Define the volumes for the 24-well source plate:

In [25]:
well_volume = 9000  # including dead volume
dead_volume = 100


In [26]:
tot_vol_water = np.sum(df_volumes['Water'].values)
print(f'Total volume of water needed: {tot_vol_water:.0f} uL + dead volume')

Total volume of water needed: 24098 uL + dead volume


### High level

In [27]:
df_stock_plate_high['Volume [uL]'] = None

for i in range(len(df_stock_plate_high)-1):
    comp = df_stock_plate_high.iloc[i]['Component']
    stock_level = 'high'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_high.loc[i, 'Volume [uL]'] = np.round(tot_vol_comp)
    
df_stock_plate_high


,Well,Component,Concentration,Volume [uL]
0,A1,MOPS[mM],2000.000,1440.0
1,B1,Tricine[mM],400.000,720.0
2,C1,H3BO3[mM],4.800,560.0
3,D1,Glucose[mM],3000.000,2870.0
4,A2,K2SO4[mM],60.000,860.0
5,B2,K2HPO4[mM],792.000,563.0
6,C2,NH4Cl[mM],1920.000,949.0
7,D2,MgCl2[mM],156.000,605.0
8,A3,NaCl[mM],3750.000,24595.0
9,B3,(NH4)6Mo7O24[mM],0.036,572.0


Are the total volumes smaller than well volume of the plate? If not, assign additional wells.

In [28]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_high.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
for i in range(len(df_stock_plate_high)-2):
    tot_volume = df_stock_plate_high.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_high.iloc[i]['Component']
        conc_level = 'high'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {
             'Component' : pd.Series(comp, index=indices),
             'Concentration': pd.Series(df_stock_plate_high.iloc[i]['Concentration'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate_high.at[i, 'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_high.drop(ind_drop, inplace=True)
df_stock_plate_high = df_stock_plate_high.append(df_new_wells, ignore_index=False)
df_stock_plate_high = df_stock_plate_high.dropna(subset=['Volume [uL]'])
df_stock_plate_high = df_stock_plate_high.sort_index().reset_index(drop=True) 


........We need 3 well(s) for NaCl[mM] with high concentration
......Finished.


/tmp/ipykernel_4848/3082395137.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new_wells = df_new_wells.append(df, ignore_index=False)
/tmp/ipykernel_4848/3082395137.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stock_plate_high = df_stock_plate_high.append(df_new_wells, ignore_index=False)


In [29]:
df_stock_plate_high


,Well,Component,Concentration,Volume [uL]
0,A1,MOPS[mM],2000.000,1540.0
1,B1,Tricine[mM],400.000,820.0
2,C1,H3BO3[mM],4.800,660.0
3,D1,Glucose[mM],3000.000,2970.0
4,A2,K2SO4[mM],60.000,960.0
5,B2,K2HPO4[mM],792.000,663.0
6,C2,NH4Cl[mM],1920.000,1049.0
7,D2,MgCl2[mM],156.000,705.0
8,NaN,NaCl[mM],3750.000,9000.0
9,B3,(NH4)6Mo7O24[mM],0.036,672.0


Assign well names:

In [30]:
well_rows = 'ABCD'
well_columns = '123456'

In [31]:
num_source_wells = len(df_stock_plate_high) 
    
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate_high['Well'] = well_names
df_stock_plate_high = df_stock_plate_high.set_index("Well")
df_stock_plate_high

,Component,Concentration,Volume [uL]
Well,,,
A1,MOPS[mM],2000.000,1540.0
B1,Tricine[mM],400.000,820.0
C1,H3BO3[mM],4.800,660.0
D1,Glucose[mM],3000.000,2970.0
A2,K2SO4[mM],60.000,960.0
B2,K2HPO4[mM],792.000,663.0
C2,NH4Cl[mM],1920.000,1049.0
D2,MgCl2[mM],156.000,705.0
A3,NaCl[mM],3750.000,9000.0


### Low level

In [32]:
df_stock_plate_low['Volume [uL]'] = None

for i in range(len(df_stock_plate_low)):
    comp = df_stock_plate_low.iloc[i]['Component']
    stock_level = 'low'
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_low.iloc[i, df_stock_plate_low.columns.get_loc('Volume [uL]')] = np.round(tot_vol_comp)

df_stock_plate_low

,Well,Component,Concentration,Volume [uL]
0,A1,H3BO3[mM],0.1200,811.0
1,B1,K2SO4[mM],3.0000,868.0
2,C1,K2HPO4[mM],79.2000,231.0
3,D1,MgCl2[mM],7.8000,791.0
4,A2,NaCl[mM],1500.0000,0.0
5,B2,(NH4)6Mo7O24[mM],0.0009,1100.0
6,C2,CoCl2[mM],0.0090,611.0
7,D2,CuSO4[mM],0.0030,3313.0
8,A3,MnSO4[mM],0.0240,764.0
9,B3,ZnSO4[mM],0.0030,574.0


In [33]:
df_new_wells = pd.DataFrame(columns=df_stock_plate_low.columns)
ind_drop = []
num_all_wells = len(df_stock_plate_high)
for i in range(len(df_stock_plate_low)):
    tot_volume = df_stock_plate_low.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate_low.iloc[i]['Component']
        conc_level = 'low'
        num_wells_needed = int(np.ceil(tot_volume / well_volume))
        print(f'We need {num_wells_needed} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = [i]
        indices.extend(range(num_all_wells, num_all_wells+num_wells_needed-1))
        num_all_wells += num_wells_needed
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {'Component' : pd.Series(comp, index=indices),
             'Concentration': pd.Series(df_stock_plate_low.iloc[0]['Concentration'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate_low.at[i,'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate_low.drop(ind_drop, inplace=True)
df_stock_plate_low = df_stock_plate_low.append(df_new_wells, ignore_index=False)
df_stock_plate_low = df_stock_plate_low.sort_index().reset_index(drop=True) 


...........Finished.


/tmp/ipykernel_4848/1564746189.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_stock_plate_low = df_stock_plate_low.append(df_new_wells, ignore_index=False)


Assign well names:

In [34]:
df_stock_plate_low = df_stock_plate_low.set_index("Well")
df_stock_plate_low

,Component,Concentration,Volume [uL]
Well,,,
A1,H3BO3[mM],0.1200,911.0
B1,K2SO4[mM],3.0000,968.0
C1,K2HPO4[mM],79.2000,331.0
D1,MgCl2[mM],7.8000,891.0
A2,NaCl[mM],1500.0000,100.0
B2,(NH4)6Mo7O24[mM],0.0009,1200.0
C2,CoCl2[mM],0.0090,711.0
D2,CuSO4[mM],0.0030,3413.0
A3,MnSO4[mM],0.0240,864.0


### Fresh components

In [35]:
df_stock_plate_fresh['Volume [uL]'] = None

comp = 'FeSO4[mM]'
for i, stock_level in enumerate(list(['low', 'high'])):
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate_fresh.loc[i+1, 'Volume [uL]'] = np.round(tot_vol_comp)

# Culture
tot_vol_culture = np.sum(df_volumes['Culture'].values) 
df_stock_plate_fresh.loc[0, 'Volume [uL]'] = np.round(tot_vol_culture)

df_stock_plate_fresh = df_stock_plate_fresh.set_index("Well")
df_stock_plate_fresh


,Component,Concentration,Volume [uL]
Well,,,
A1,Culture,NaN,720.0
B1,FeSO4[mM],0.3,478.0
C1,FeSO4[mM],6.0,543.0


Save source plate instructions:

In [36]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_high.csv"
df_stock_plate_high.to_csv(stock_plate_file)

In [37]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_low.csv"
df_stock_plate_low.to_csv(stock_plate_file)

In [38]:
stock_plate_file = f"{user_params['output_path']}/24-well_stock_plate_fresh.csv"
df_stock_plate_fresh.to_csv(stock_plate_file)

## Define transfers for biomek

In [39]:
column_names = [
    "Source Position",
    "Source Well",
    "Destination Position",
    "Destination Well",
    "Transfer Volume [uL]"
]

### Create water transfers

In [40]:
P20_water = pd.DataFrame(columns=column_names)
P200_water = P20_water.copy()

i = 0  # counter for index

comp = 'Water'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_water.at[i, "Destination Well"] = dest_well
        P20_water.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_water.at[i, "Destination Well"] = dest_well
            P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_water["Source Position"] = "P3" 
P20_water["Source Well"] = "A1"  # reservoir plate
P20_water["Destination Position"] = "P2" 
P200_water["Source Position"] = "P3" 
P200_water["Source Well"] = "A1"
P200_water["Destination Position"] = "P2" 

### Create kan transfers

In [42]:
P20_kan = pd.DataFrame(columns=column_names)
P200_kan = P20_kan.copy()

i = 0  # counter for index

comp = 'Kan[g/L]'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_kan.at[i, "Destination Well"] = dest_well
        P20_kan.at[i, "Transfer Volume [uL]"] = vol
        i += 1

    # P200 transfer
    else:
        # Divide the transfer in parts until whole volume is transfered
        # How many transfers
        num_transfers = int(np.ceil(vol / max_tip_volume))
        transf_vol = vol / num_transfers
        for trans_num in range(num_transfers):
            P200_kan.at[i, "Destination Well"] = dest_well
            P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
            i += 1

P20_kan["Source Position"] = "P1" 
P20_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P20_kan["Destination Position"] = "P2"
P200_kan["Source Position"] = "P1" 
P200_kan["Source Well"] = df_stock_plate_high[
    df_stock_plate_high["Component"]==comp
].index[0]  
P200_kan["Destination Position"] = "P2"


### Create culture transfers

Note that we assume here the volumes for culture are small enough for p20 pipette, if that is not the case you need to adjust.

In [43]:
P20_culture = pd.DataFrame(columns=column_names)

comp = 'Culture'
vol = df_volumes[comp][0]
    
for i, dest_well in enumerate(df_volumes.index):
    
    P20_culture.at[i, "Destination Well"] = dest_well
    P20_culture.at[i, "Transfer Volume [uL]"] = vol

# DBTL 1
# P20_culture["Source Position"] = "P4" 
# P20_culture["Source Well"] = df_stock_plate_low[
#     df_stock_plate_low["Component"]==comp
# ].index[0]

P20_culture["Source Position"] = "P5" 
P20_culture["Source Well"] = df_stock_plate_fresh[
    df_stock_plate_fresh["Component"]==comp
].index[0]
P20_culture["Destination Position"] = "P2" 


### Create component transfers

Create a column to track the current volume in the wells of the source plate:

In [44]:
df_stock_plate_high['Current Volume'] = well_volume
df_stock_plate_low['Current Volume'] = well_volume
df_stock_plate_fresh['Current Volume'] = well_volume

In [45]:
P20_components = pd.DataFrame(columns=column_names) 
P200_components = P20_components.copy()

In [46]:
df_volumes.columns

Index(['MOPS[mM]', 'Tricine[mM]', 'H3BO3[mM]', 'Glucose[mM]', 'K2SO4[mM]',
       'K2HPO4[mM]', 'FeSO4[mM]', 'NH4Cl[mM]', 'MgCl2[mM]', 'NaCl[mM]',
       '(NH4)6Mo7O24[mM]', 'CoCl2[mM]', 'CuSO4[mM]', 'MnSO4[mM]', 'ZnSO4[mM]',
       'CaCl2[mM]', 'Kan[g/L]', 'Water', 'Culture'],
      dtype='object')

In [47]:
i = 0  # counter for index

components = list(df_volumes.columns.drop(['Kan[g/L]', 'Water', 'Culture']))

for comp in components:
    
    for dest_well in df_volumes.index:
        vol = df_volumes.at[dest_well, comp]
        conc_level = df_conc_level.at[dest_well, comp]
        
        # Find which plate, well we need to use as source
        if comp == 'FeSO4[mM]':
            plate_position = "P5"
            df_stock_plate = df_stock_plate_fresh
            source_well = "B1" if conc_level == 'low' else "C1"
        else:
            if conc_level == 'high':
                plate_position = "P1" 
                df_stock_plate = df_stock_plate_high
            else:
                plate_position = "P4"
                df_stock_plate = df_stock_plate_low

            # Find the source well in the stock plate with enough volume 
            # Find all wells with this component
            source_wells = df_stock_plate[
                (df_stock_plate["Component"]==comp)
            ].index

            # Find indices where these wells have volume larger than transfer volume
            ind_vol = np.where(df_stock_plate.loc[source_wells, 'Current Volume']  > vol + dead_volume)[0]
            # Assign as the source well the first one with enough volume
            source_well = source_wells[ind_vol[0]]    
        
        # P20 transfer
        if vol < 30:
            P20_components.at[i, "Source Well"] = source_well
            P20_components.at[i, "Destination Well"] = dest_well
            P20_components.at[i, "Transfer Volume [uL]"] = vol
            P20_components.at[i, "Source Position"] = plate_position
            i += 1

        # P200 transfer
        else:
            # Divide the transfer in parts until whole volume is transfered
            # How many transfers
            num_transfers = int(np.ceil(vol / max_tip_volume))
            transf_vol = vol / num_transfers
            for trans_num in range(num_transfers):
                P200_components.at[i, "Source Well"] = source_well
                P200_components.at[i, "Destination Well"] = dest_well
                P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                P200_components.at[i, "Source Position"] = plate_position
                i += 1
                    
        # Update the current volume in the source well
        df_stock_plate.loc[source_well, 'Current Volume'] -= vol

P20_components["Destination Position"] = "P2" 
P200_components["Destination Position"] = "P2" 

In [48]:
P20_components.head(5)

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
0,P1,A1,P2,A1,30.0
1,P1,A1,P2,B1,30.0
2,P1,A1,P2,C1,30.0
6,P1,A1,P2,A2,30.0
7,P1,A1,P2,B2,30.0


## Calculate number of transfers and tips needed

In [49]:
num_transf_water_p200 = len(P200_water)
num_transf_water_p20 = len(P20_water)
num_transf_comp_p20 = len(P20_components)
num_transf_comp_p200 = len(P200_components)
num_transf_culture = len(P20_culture)
num_transf_kan_p20 = len(P20_kan)
num_transf_kan_p200 = len(P200_kan)
num_mixing = 1  # mixing the culture well 
num_total_transf = num_transf_water_p200 + num_transf_water_p20 + num_transf_comp_p20 + num_transf_comp_p200+num_transf_kan_p20+num_transf_kan_p200+num_transf_culture+num_mixing
print(f'Number of transfers:')
print(f'\t Water (p200): {num_transf_water_p200}')
print(f'\t Water (p20): {num_transf_water_p20}')
print(f'\t Kan (p20): {num_transf_kan_p20}')
print(f'\t Kan (p200): {num_transf_kan_p200}')
print(f'\t Components (p200): {num_transf_comp_p200}')
print(f'\t Components (p20): {num_transf_comp_p20}')
print(f'\t Culture (p20): {num_transf_culture}')
print(f'\t Mixing (p20): {num_mixing}')
print(f'Total number of transfers: {num_total_transf}')

Number of transfers:
	 Water (p200): 150
	 Water (p20): 3
	 Kan (p20): 48
	 Kan (p200): 0
	 Components (p200): 312
	 Components (p20): 573
	 Culture (p20): 48
	 Mixing (p20): 1
Total number of transfers: 1135


Calculate number of tip boxes needed:

In [50]:
print(f'This protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_comp_p20 + num_transf_kan_p20 + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')


This protocol requires:
	4 box(es) of p200 tips
		3 box(es) + 31 tips
	8 box(es) of p20 tips
		7 box(es) + 5 tips


Introduce a pause in the biomek method after p200 components transfers.

In [51]:
print(f'The first part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200 
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_kan_p20 

full_box, rest = divmod(num_tips_200, 96)
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The first part of the protocol requires:
	4 box(es) of p200 tips
		3 box(es) + 31 tips
	1 box(es) of p20 tips
		0 box(es) + 55 tips


For the next step we need:

In [52]:
print(f'The second part of the protocol requires:')
# For water transfers (from a reservoir plate) biomek is using only 7 probes
num_tips_20 = num_transf_comp_p20  + num_mixing + num_transf_culture
full_box, rest = divmod(num_tips_20, 96)
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')
print(f'\t\t{full_box} box(es) + {rest} tips')

The second part of the protocol requires:
	7 box(es) of p20 tips
		6 box(es) + 46 tips


### Save biomek files

In [53]:
biomek_files_dir = f"{user_params['output_path']}/biomek_files/"
os.makedirs(biomek_files_dir, exist_ok=True)

P200_water_file = f"{biomek_files_dir}/P200_water.csv"
P20_water_file = f"{biomek_files_dir}/P20_water.csv"
P20_kan_file = f"{biomek_files_dir}/P20_kan.csv"
P200_components_file = f"{biomek_files_dir}/P200_components.csv"
P20_components_file = f"{biomek_files_dir}/P20_components.csv"
P20_culture_file = f"{biomek_files_dir}/P20_culture.csv"

P200_water.to_csv(P200_water_file, index=False)
P20_water.to_csv(P20_water_file, index=False)
P20_kan.to_csv(P20_kan_file, index=False)
P200_components.to_csv(P200_components_file, index=False)
P20_components.to_csv(P20_components_file, index=False)
P20_culture.to_csv(P20_culture_file, index=False)